# Create and run Earthquake Estimator

In [1]:
!nvidia-smi

Wed Jan 30 08:29:05 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
import lstm_estimator_4096
from importlib import reload
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
import pickle
%matplotlib inline

In [3]:
def reload_tf():
    global lstm_estimator
    global earthquake_input_fn
    global lstm_estimator_4096
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        import lstm_estimator_4096
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        reload(lstm_estimator_4096)
        del lstm_estimator
        del earthquake_input_fn
        del lstm_estimator_4096
    import lstm_estimator
    import earthquake_input_fn
    import lstm_estimator_4096
reload_tf()

In [4]:
def create_estimator_4096_from_params(batch_size, timesteps, n_feats, feature_columns,
                                      lstm_cell_size, dense_size, cnn_size, learning_rate, dropout_rate=0.5, grad_clip=0.5,
                                      time_pool=8, lambda_l2_reg=0.0005, stft_frame_length=512, stft_frame_step=64,
                                      label_input_column=None, model_dir=None, eval_every=600):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'n_feats': n_feats,
        'feature_columns': feature_columns,
        'lstm_cell_size': lstm_cell_size,
        'dense_size': dense_size,
        'cnn_size': cnn_size,
        'learning_rate': learning_rate,
        'dropout_rate': dropout_rate,
        'grad_clip': grad_clip,
        'lambda_l2_reg': lambda_l2_reg,
        'stft_frame_length': stft_frame_length,
        'stft_frame_step': stft_frame_step,
        'label_input_column': label_input_column,
        'time_pool': time_pool
    }
    if model_dir is None:
        model_dir = '/workspace/persistent-data/models/%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
    
    params_file = osp.join(osp.split(model_dir)[0], osp.split(model_dir)[1] + '.params.pickle')
    if not osp.isfile(params_file):
        print('writing params to %s' % params_file)
        with open(params_file, 'wb') as f:
            pickle.dump(params, f)
        
    config = tf.estimator.RunConfig(model_dir=model_dir,
                                    log_step_count_steps=int(4096 / timesteps) * 10,
                                    save_checkpoints_secs=300)
    
    estim = tf.estimator.Estimator(model_fn=lstm_estimator_4096.lstm_4096_model_fn,
                                   params=params,
                                   model_dir=model_dir,
                                   config=config)
    return estim

In [5]:
def train_and_evaluate_estimator_4096(earthquake_data_dir, scales, eval_every, eval_count, batch_size,
                                      timesteps, n_feats, feature_columns, lstm_cell_size, dense_size, cnn_size,
                                      learning_rate, dropout_rate, grad_clip, time_pool, lambda_l2_reg,
                                      stft_frame_length, stft_frame_step, model_dir=None):
    
    estim = create_estimator_4096_from_params(batch_size, timesteps, n_feats, feature_columns=feature_columns,
                                              lstm_cell_size=lstm_cell_size, dense_size=dense_size, cnn_size=cnn_size,
                                              learning_rate=learning_rate, dropout_rate=dropout_rate, grad_clip=grad_clip,
                                              time_pool=time_pool, lambda_l2_reg=lambda_l2_reg, stft_frame_length=stft_frame_length,
                                              stft_frame_step=stft_frame_step, label_input_column=None, model_dir=model_dir,
                                              eval_every=eval_every)
    
    trainspec = tf.estimator.TrainSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                             batch_size,
                                                                             timesteps,
                                                                             scales=scales,
                                                                             traintest='train'),
                                       max_steps=1000000)
    
    evalspec = tf.estimator.EvalSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                           batch_size,
                                                                           timesteps,
                                                                           scales=scales,
                                                                           traintest='test'),
                                     steps=int(eval_count * (4096/timesteps)),
                                     start_delay_secs=1, throttle_secs=eval_every)
    
    return estim, trainspec, evalspec

In [25]:
reload_tf()

EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords2'
BATCH_SIZE=32
TIMESTEPS=4096
LSTM_CELL_SIZE=[512, 512, 512]
CNN_SIZE=[[32, 4], [32, 4], [16, 6], [12, 6], [8, 8]]
DENSE_SIZE=[128, 4]
STFT_FRAME_LENGTH=256
STFT_FRAME_STEP=32
LEARNING_RATE=0.001
DROPOUT_RATE=0.5
GRAD_CLIP=1
TIME_POOL=32
LAMBDA_L2_REG=0.001
STEPS_PER_BATCH = int(4096/TIMESTEPS)
EVAL_EVERY_N_SECONDS = 600
EVAL_NUM_BATCHES = 300
N_FEATS = 1
SCALES = ['1e' + str(i) for i in range(-8,3)]
FEATURE_COLUMNS = [tf.feature_column.numeric_column(key='acousticdata', dtype=tf.float32, shape=(TIMESTEPS,))]

estim, train_spec, eval_spec = train_and_evaluate_estimator_4096(EARTHQUAKE_DATA_DIR, SCALES, EVAL_EVERY_N_SECONDS,
                                                                 EVAL_NUM_BATCHES, BATCH_SIZE, TIMESTEPS, N_FEATS, FEATURE_COLUMNS,
                                                                 LSTM_CELL_SIZE, DENSE_SIZE, CNN_SIZE, LEARNING_RATE, DROPOUT_RATE, GRAD_CLIP,
                                                                 TIME_POOL, LAMBDA_L2_REG, STFT_FRAME_LENGTH, STFT_FRAME_STEP,
                                                                 #model_dir='/workspace/persistent-data/models/2019-01-30-08-36-04'
                                                                 )

writing params to /workspace/persistent-data/models/2019-01-30-09-01-13.params.pickle
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_master': '', '_log_step_count_steps': 10, '_keep_checkpoint_max': 5, '_task_id': 0, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0116d59c88>, '_is_chief': True, '_num_worker_replicas': 1, '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_task_type': 'worker', '_train_distribute': None, '_device_fn': None, '_eval_distribute': None, '_evaluation_master': '', '_model_dir': '/workspace/persistent-data/models/2019-01-30-09-01-13', '_num_ps_replicas': 0, '_protocol': None, '_save_checkpoints_secs': 300, '_service': None}


In [ ]:
for i in range(1):
    try:
        print('---------------- BEGIN EPOCH %d ---------------' % i)
        tf.estimator.train_and_evaluate(estim, train_spec, eval_spec)
    except KeyboardInterrupt:
        break
    #except:
    #    print('end of epoch')

---------------- BEGIN EPOCH 0 ---------------
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
FEATURES
{'acousticdata': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?) dtype=float32>}
--------------------
LABELS
Tensor("IteratorGetNext:1", shape=(?, ?), dtype=float64, device=/device:CPU:0)
--------------------
MODE
train
--------------------
PARAMS
{'n_feats': 1, 'timesteps': 4096, 'dense_size': [128, 4], 'cnn_size': [[32, 4], [32, 4], [16, 6], [12, 6], [8, 8]], 'lambda_l2_reg': 0.001, 'label_input_column': None, 'grad_clip': 1, 'batch_size': 32, 'stft_frame_length': 256, 'time_pool': 32, 'feature_columns': [_NumericColumn(key='acousticdata', shape=(4096,), default